In [1]:
from SL_C0de.SOLVER import Precomputation

from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Loading Ice data

data=io.loadmat(r'H:/SL_C0de/SL_C0de_data/ice6G122k') #load the file, can be modified to load an other file.
ice_in=data['ice6g']# ice_in[temps, latitude, longitude] hauteur de la glace
ice_time=data['ice_time']
ice_time=ice_time.squeeze()
ice_time=np.append(ice_time[0]+1,ice_time)
ice_lon =data['ice_long'].squeeze()
ice_lat =data['ice_lat'].squeeze()[::-1]
# The ice is not in the good shape. We have to derivate it : 
ice_0=ice_in.T[0,:,:].copy()
ice_in=np.diff(ice_in.T,axis=0)
ice_in=np.concatenate((np.expand_dims(ice_0,axis=0),ice_in),axis=0)

ice_grid=dict(name='ICE_ICE6G', grid=ice_in, time=ice_time,lon=ice_lon,lat=ice_lat)

# Loading sediment loading data

sed_ncgrid = pd.read_csv(r'H:/SL_C0de/SL_C0de_data/AYS2_sed.csv') 

lon=np.array(sed_ncgrid["POINT_X.N.19.11"])
lat=np.array(sed_ncgrid["POINT_Y.N.19.11"])
sed=np.array(sed_ncgrid[:]).T/4
sed=sed[3:,:]
#sed=sed[::-1,:]*0 # add a zero sediment model
sed_time_step=np.arange(26.25,-0.25,-0.25)

area=(93,98,13,20)

sed_grid=dict(name='AYS2', grid=sed, time=sed_time_step,lon=lon,lat=lat,frame=area)

# Loading topographic data

data=io.loadmat(r'H:/SL_C0de/SL_C0de_data/topo_SL') #load the file, can be modified to load an other file.
topo_grid=data['topo_bed'][::2,1::2].squeeze()
topo_lat=data['lat_topo'].squeeze()[::-2]
topo_lon=data['lon_topo'].squeeze()[1::2]

topo_grid=dict(name='topo_SL',grid=topo_grid,lon=topo_lon,lat=topo_lat)




In [6]:
# Precompute the full data to prepare the next resolution of the SLE

stop=26 # define the number of time steps
step=0.5
time_step=np.arange(start=stop,stop=-step,step=-step) # in kyr to present
maxdeg=512 # define the maximum degree for spherical harmonics.
Output_way='H:/SL_C0de/Interpolated_grid/Test_512_no_sed'
plot=True

Precomputation(ice_grid,sed_grid,topo_grid,Output_way,stop=stop,step=step,maxdeg=maxdeg,plot=plot)


In [1]:
# Solve the SLE in forward modeling way 
from SL_C0de.SOLVER import SLE_forward_modeling

Input_way='H:/SL_C0de/Interpolated_grid/Test'
ice_name='ICE_ICE6G'
sediment_name='AYS2'
topo_name='topo_SL'
ocean_name='OCE'
love_way='H:/SL_C0de/SL_C0de_data/variable_lm_32'
love_file='VM5a.l32.um21.lm22.449'
conv_lim=10e-10
Output_way='H:/SL_C0de/Interpolated_grid/Test'

SLE_forward_modeling(Input_way,ice_name,sediment_name,topo_name,ocean_name,love_way,love_file,conv_lim,Output_way)

(6,) (6,) (21,) (21,) (490, 6) (490, 6)


UnboundLocalError: cannot access local variable 't_it' where it is not associated with a value

In [1]:
from SL_C0de.SOLVER import SLE_solver

Input_way='H:/SL_C0de/Interpolated_grid/Test_512'
ice_name='ICE_ICE6G'
sediment_name='AYS2'
topo_name='topo_SL'
ocean_name='OCE'
love_way='H:/SL_C0de/SL_C0de_data/'
love_file='PREM.l50.ast8e19.um1.8e20.lm50e21'
conv_lim=10e-10
topo_lim=3
Output_way='H:/SL_C0de/Interpolated_grid/Test_512'

load=SLE_solver(Input_way,ice_name,sediment_name,topo_name,ocean_name,love_way,love_file,topo_lim,conv_lim,Output_way)

time_iteration :  25.5
13
time_iteration :  25.0
12
time_iteration :  24.5
12
time_iteration :  24.0
12
time_iteration :  23.5
12
time_iteration :  23.0
12
time_iteration :  22.5
11
time_iteration :  22.0
11
time_iteration :  21.5
12
time_iteration :  21.0
12
time_iteration :  20.5
12
time_iteration :  20.0
11
time_iteration :  19.5
11
time_iteration :  19.0
11
time_iteration :  18.5
11
time_iteration :  18.0
11
time_iteration :  17.5
11
time_iteration :  17.0
11
time_iteration :  16.5
12
time_iteration :  16.0
11
time_iteration :  15.5
11
time_iteration :  15.0
11
time_iteration :  14.5
11
time_iteration :  14.0
11
time_iteration :  13.5
11
time_iteration :  13.0
11
time_iteration :  12.5
11
time_iteration :  12.0
11
time_iteration :  11.5
11
time_iteration :  11.0
11
time_iteration :  10.5
11
time_iteration :  10.0
12
time_iteration :  9.5
11
time_iteration :  9.0
11
time_iteration :  8.5
9
time_iteration :  8.0
11
time_iteration :  7.5
11
time_iteration :  7.0
12
time_iteration :  6

In [2]:
from SL_C0de.SOLVER import Post_process

Input_way='H:/SL_C0de/Interpolated_grid/Test_512/model_output'
sed_name='AYS2'
ice_name='ICE_ICE6G'
ocean_name='OCE'
topo_name='topo_SL'
love_way='H:/SL_C0de/SL_C0de_data'
Post_process(Input_way,sed_name,ice_name,ocean_name,topo_name,love_way)

calculation for : PREM.l32.ast8e19.um1.8e20.lm50e21
Oceanic sediment calculation
calculation for : PREM.l50.ast8e19.um1.8e20.lm50e21
Oceanic sediment calculation


In [4]:
import numpy as np
Input_way='H:/SL_C0de/Interpolated_grid/Test_512_no_sed/model_output/VM5A.l32.um20.25.lm22.699'
plot=dict(plot=True,
          times=[0],
          frames=[(-179,179,-89,89),(93,98,13,20)],
          frames_resolution=[1024,2048],
          #frames_min_max=np.array([[-30,200,-3,20],[-2,3,-0.2,0.1]]),
          frames_min_max=np.array([[None,None,None,None],[None,None,None,None]]),
          contours_v=[[[i for i in range(-30,-10,10)]+[i for i in range(-10,10,1)]+[i for i in range(10,30+1,10)]+[100,200],
                       [i/10 for i in range(-18,0,1)]+[i/20 for i in range(0,10,1)]],
                       [[-3,-0.1,0,0.1,3,10,20],[-0.6,-0.3,-0.2,-0.15,-0.1,0]]],
          #contours_v=[[[None],[None]],[[None],[None]]],
          transects=[(21.63,96.01,11.1235,96.04),(15.347829, 89.811065,15.531321, 99.098801)],
          point_density=[512,512],
          transect_min_max=[(-2,0.5,-0.7,1),(-3,2,-0.7,1)],
          points=np.array([[16.8,96],[15,96.25],[15.9609036,95.4758372],[16.0318760,94.8894641],[16.3469326,95.2544328]]),
        )

In [5]:
from SL_C0de.SOLVER import plot_model_result_map
plot_model_result_map(Input_way,plot)

c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!
c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!
c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!
c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!
c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a*

In [5]:
from SL_C0de.SOLVER import plot_model_result_cross_section
plot_model_result_cross_section(Input_way,plot)

c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!


In [6]:
from SL_C0de.SOLVER import plot_model_output_points
plot_model_output_points(Input_way,plot)

c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!


In [6]:
from SL_C0de.SOLVER import export_to_netcdf 
export_to_netcdf(Input_way,0,plot['frames_resolution'][1],plot['frames'][1],Input_way+'\LOAD','SEDIMENT','LOAD')


In [8]:
from SL_C0de.grid import LOAD_TIME_GRID
Input_way="H:\SL_C0de\Interpolated_grid\Test_512\model_output\VM5a.l32.um20.25.lm22.699\LOAD\TOTAL_GEOID"
geoid_grid=LOAD_TIME_GRID(from_file=(True,Input_way))

c:\Users\ahenry01\Desktop\Python_code\SL_C0de_lib_0_4_0\src\SL_C0de\love.py:51: RuntimeWarning: invalid value encountered in divide
  const = 4*np.pi*a**3/Me # setting the tide constant for earth !!!! CHECK !!!!


In [9]:
import numpy as np
geoid_grid.height_time_coeff=geoid_grid.elastic_deformation+geoid_grid.viscuous_deformation
geoid_grid.point_time(np.array([[15,96.25]]))

array([[-0.04113322, -0.05264783,  0.25859711,  0.12994494,  0.02530244,
         0.06228279,  0.09552334,  0.09933712,  0.11552406,  0.10993514,
         0.08122948,  0.06044233,  0.05826695,  0.06562002,  0.06344963,
         0.11611301,  0.12875095,  0.10382271,  0.08306646,  0.16361093,
         0.2168013 ,  0.31669921,  0.25768939,  0.16710384,  0.77414759,
         1.41883381,  1.08117731,  0.68155103,  0.63412163,  0.6085443 ,
         0.84066766,  1.02052021,  0.83835957,  0.6634248 ,  0.59466263,
         0.52192084,  0.44075518,  0.52075407,  0.39689737,  0.17865839,
         0.06941568,  0.00765629, -0.01033691, -0.0144748 , -0.02718773,
        -0.0387622 , -0.05255559, -0.04533695, -0.03065008, -0.0223101 ,
        -0.03014018, -0.04113322, -0.05264783]])

In [7]:
from SL_C0de.SOLVER import calculate_dESL
Input_way="H:\SL_C0de\Interpolated_grid\Test_512\model_output\VM5a.l32.um20.25.lm22.699"
calculate_dESL(Input_way)

array([-2.14167422e+02,  2.06687523e+00,  2.22026885e+00,  1.97151656e+00,
        1.24028209e+00,  1.24503995e+00,  1.26878855e+00,  1.15142975e+00,
        8.11376229e-01,  7.71836635e-01,  9.46768617e-01,  9.66866436e-01,
        1.07723290e+00,  1.02838670e+00,  1.18300725e+00,  1.66229261e+00,
        1.99199559e+00,  1.60163140e+00,  1.94167161e+00,  2.15162222e+00,
        4.19870988e+00,  3.59992786e+00,  2.02780624e+00,  1.39704622e+01,
        1.49221196e+01,  5.69409710e+00,  5.18560551e+00,  5.18961201e+00,
        4.68364468e+00,  7.99335744e+00,  8.10913174e+00,  5.34445328e+00,
        5.02959284e+00,  4.63867466e+00,  3.99914788e+00,  3.43498247e+00,
        4.13747931e+00,  1.41230096e+00,  8.94019760e-01,  3.07973453e-01,
        2.49363993e-02,  6.68025061e-02,  4.72132343e-02,  5.06866965e-02,
        6.96483234e-02, -1.45251653e-02,  1.82474734e-02,  2.55972894e-03,
       -1.81729987e-02, -7.55220818e-03, -1.43834587e-02, -7.23403656e-03,
        0.00000000e+00])